In [1]:
# for each ligand, generates ligand descriptor & coordinates of the voxels in the descriptor
# each pair of data (descriptor, centers) is saved as a pickle file

# outputs a csv containing ligand id & pickle path

In [3]:
# generate ligand descriptor files + csv 
from htmd.ui import *
from htmd.molecule.voxeldescriptors import *
from pathlib import Path
import numpy as np
import pickle as pk
from multiprocessing import Pool
import pandas as pd
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/
To update: conda update htmd -c acellera -c psi4

You are on the latest HTMD version (1.13.9).



In [4]:
#Supress default INFO logging

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [5]:
ligands = []
ligand_root = '../processed_data/ligands'
Path(ligand_root).mkdir(exist_ok = True, parents=True)

/home/alex/miniconda3/lib/python3.6/site-packages/pyemma/__init__.py:91: UserWarning: You are not using the latest release of PyEMMA. Latest is 2.5.2, you have 2.4.
  .format(latest=latest, current=current), category=UserWarning)


In [6]:
raw_ligand_paths = sorted(list(Path('../data/filtered_data').glob('*lig_cg.pdb')))

In [7]:
dests = []
for idx, lig_path in tqdm(enumerate(raw_ligand_paths)):
    mol = Molecule(str(lig_path), keepaltloc='all')
    bb = htmd.molecule.util.boundingBox(mol)

    xx = (bb[1][0] + bb[0][0])/2 - 12
    yy = (bb[1][1] + bb[0][1])/2 - 12
    zz = (bb[1][2] + bb[0][2])/2 - 12

    centers = []
    for ix in range(24):
        for iy in range(24):
            for iz in range(24):
                centers.append([xx + ix, yy + iy, zz + iz])

    features, centers = getVoxelDescriptors(mol, usercenters=np.array(centers), voxelsize=1, method='CUDA')
    features = features.reshape(24, 24, 24, features.shape[1])
    features = features[:,:,:,0]
    
    dest = ligand_root + '/' + str(idx+1).zfill(4) + '.pk'
    with open(dest, 'wb') as f:
        pk.dump((features, centers), f)
        dests.append(dest)
        #print('done with '+str(idx+1))

0it [00:00, ?it/s]/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
1it [00:00,  1.88it/s]/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
3000it [03:07, 16.00it/s]


In [8]:
idxs = list(range(1,3001))
df = pd.DataFrame({'id':idxs, 'path': dests})
df.to_csv('ligand_data.csv', index=None)